In [24]:
import pandas as pd

# Read the data from the Excel file
data = pd.read_excel('course_data3.xlsx')



In [25]:
# Count the unique occurrences in the "Category" column
category_counts = data['Category'].value_counts()

# Print the unique occurrences and their counts
print(category_counts)


Essential units    1061
Elective units      667
Major                44
Specialisation       30
Capstone unit        28
Name: Category, dtype: int64


In [26]:
# Group units containing "elective" into "Elective units" category
data.loc[data['Category'].str.contains('elective', case=False), 'Category'] = 'Elective units'

# Group units containing "essential" into "Essential units" category
data.loc[data['Category'].str.contains('essential', case=False), 'Category'] = 'Essential units'

# Count the unique occurrences in the updated "Category" column
category_counts = data['Category'].value_counts()

# Print the unique occurrences and their counts
print(category_counts)


Essential units    1061
Elective units      667
Major                44
Specialisation       30
Capstone unit        28
Name: Category, dtype: int64


In [27]:
filtered_data = data[data['Category'].isin(['Major', 'Specialisation', 'Capstone unit'])]

# Create a new DataFrame for the filtered rows
filtered_df = pd.DataFrame(filtered_data)

# Print the new DataFrame
print(filtered_df)


                                         Degree       Category Course Code  \
31                         Bachelor of Business  Capstone unit    MQBS3020   
32                         Bachelor of Business          Major     N000191   
33                         Bachelor of Business          Major     N000192   
34                         Bachelor of Business          Major     N000276   
35                         Bachelor of Business          Major     N000277   
...                                         ...            ...         ...   
1547         Bachelor of Information Technology          Major     N000086   
1548         Bachelor of Information Technology          Major     N000272   
1549         Bachelor of Information Technology          Major     N000062   
1550         Bachelor of Information Technology          Major     N000063   
1567  Bachelor of Biodiversity and Conservation  Capstone unit    BIOL3420   

                                    Course Name  
31           

In [31]:
# Filter rows based on "credit point" in "Course Code" column
data = data[~data['Course Code'].str.contains('credit point', case=False)]

# Reset the index after dropping rows
data = data.reset_index(drop=True)

In [32]:
# Group units containing "elective" into "Elective units" category
data.loc[data['Category'].str.contains('elective', case=False), 'Category'] = 'Elective units'

# Group units containing "essential" into "Essential units" category
data.loc[data['Category'].str.contains('essential', case=False), 'Category'] = 'Essential units'

# Separate majors and specializations into two lists
majors = list(data[data['Category'] == 'Major']['Course Code'])
specializations = list(data[data['Category'] == 'Specialisation']['Course Code'])

# Print the majors and specializations lists
print("Majors:")
print(majors)
print("\nSpecializations:")
print(specializations)

Majors:
['N000191', 'N000192', 'N000276', 'N000277', 'N000278', 'N000279', 'N000184', 'N000185', 'N000186', 'N000187', 'N000188', 'N000190', 'N000191', 'N000192', 'N000279', 'N000280', 'N000261', 'N000255', 'N000259', 'N000258', 'N000264', 'N000256', 'N000262', 'N000260', 'N000114', 'N000263', 'N000111', 'N000171', 'N000172', 'N000173', 'N000113', 'N000083', 'N000013', 'N000027', 'N000029', 'N000267', 'N000271', 'N000054', 'N000055', 'N000086', 'N000272', 'N000062', 'N000063']

Specializations:
['R000269', 'R000265', 'R000266', 'R000270', 'R000161', 'R000162', 'R000267', 'R000160', 'R000268', 'R000271', 'R000159', 'R000163', 'R000164', 'R000124', 'R000125', 'R000128', 'R000262', 'R000263', 'Q000230', 'Q000105', 'Q000106', 'Q000107', 'Q000110', 'Q000109', 'Q000233', 'Q000235', 'Q000236', 'R000081', 'R000141', 'R000253']


In [34]:
all_majors = majors + specializations

In [33]:
# data.to_excel('course_data3.xlsx', index=False)

In [36]:
import pandas as pd
from selenium import webdriver
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException
import collections
from selenium.common.exceptions import TimeoutException
import pandas as pd
import requests
import time

firefox_option = Options()
firefox_option.add_argument("--headless")
driver = webdriver.Firefox(options=firefox_option)

In [80]:
all_missing = []
all_ess = []
all_elec = []
all_title =[]

In [81]:
for major in all_majors:
    ess = []
    elec = []
    url = f"https://coursehandbook.mq.edu.au/2023/aos/{major}"
    driver.get(url)
    time.sleep(0.3)
    html = driver.page_source
    # Parse the HTML content using BeautifulSoup
    soup = BeautifulSoup(html, "html.parser")

    if driver.current_url == "https://coursehandbook.mq.edu.au/404-page-not-found":
        url = f"https://coursehandbook.mq.edu.au/2022/aos/{major}"
        driver.get(url)
        time.sleep(1)
        if driver.current_url == "https://coursehandbook.mq.edu.au/404-page-not-found":
            all_missing.append(major)
            all_ess.append("N/A")
            all_elec.append("N/A")
            all_title.append("N/A")
            continue
    
    # Major title
    div_academic_banner = soup.find('div', id='academic-item-banner').find("h2")
    # Print the text of the <h2> element
    title = div_academic_banner.text.strip()
    div_curriculum_structure = soup.find('div', id='curriculumStructure')
    div_accordion_container = div_curriculum_structure.find_all('div', {'name': 'ACCORDION_CONTAINER'})
    for div in div_accordion_container:
        essential_units = div.find('strong')
        div_subjects = div.find_all('div', {'filter': 'subject'})
        for div2 in div_subjects:
            div_grey_500 = div2.find_all('div', {'color': 'grey._500'})
            if "essential" in essential_units.text.lower():
                ess.append(div_grey_500[0].text)
            elif "elective" in essential_units.text.lower():
                elec.append(div_grey_500[0].text)
    print(title)
    print(ess)
    print(elec)
    print(all_missing)
    print("------------------")

    all_ess.append(ess)
    all_elec.append(elec)
    all_title.append(title)

Human Resource Management
['MGMT1007', 'MGMT2022', 'MGMT2040', 'MGMT2050', 'MGMT3007', 'MGMT3015', 'MGMT3028']
['MGMT3000', 'MGMT3017']
[]
------------------
International Business
['MGMT1005', 'MGMT2002', 'MGMT2030', 'MGMT2031', 'MGMT3001']
['BUSA3015', 'ECON3020', 'MGMT3000', 'MQBS3030']
[]
------------------
Accounting
['ACCG1001', 'ACCG2000', 'ACCG2024', 'ACCG2050', 'ACCG3008']
['ACCG3001', 'ACCG3030', 'ACCG3050', 'ACCG3055']
['N000276', 'N000277', 'N000278', 'N000279']
------------------
Business Analytics
['BUSA1000', 'ACCG2050', 'BUSA2020', 'STAT2170', 'ACCG3060', 'BUSA3020', 'STAT3102']
['ACCG3055', 'BUSA3015']
['N000276', 'N000277', 'N000278', 'N000279']
------------------
Business Information Systems
['COMP1350', 'ACCG2050', 'COMP2350', 'COMP2750', 'ACCG3025', 'ACCG3055', 'ACCG3058', 'COMP3770']
[]
['N000276', 'N000277', 'N000278', 'N000279']
------------------
Cyber Security Governance
['COMP1300', 'ACCG2050', 'ACCG2065', 'PICT2001', 'ACCG3025', 'ACCG3058', 'ACCG3060', 'PICT

In [82]:
# major = "N000055"
# ess = []
# elec = []
# url = f"https://coursehandbook.mq.edu.au/2023/aos/{major}"
# driver.get(url)
# time.sleep(0.3)
# html = driver.page_source
# # Parse the HTML content using BeautifulSoup
# soup = BeautifulSoup(html, "html.parser")

# # Major title
# div_academic_banner = soup.find('div', id='academic-item-banner').find("h2")


# # Print the text of the <h2> element
# title = div_academic_banner.text.strip()

# if driver.current_url == "https://coursehandbook.mq.edu.au/404-page-not-found":
#     url = f"https://coursehandbook.mq.edu.au/2022/aos/{major}"
#     driver.get(url)
#     if driver.current_url == "https://coursehandbook.mq.edu.au/404-page-not-found":
#         all_missing.append(major)
#         all_ess.append("N/A")
#         all_elec.append("N/A")
#         all_title.append("N/A")
# div_curriculum_structure = soup.find('div', id='curriculumStructure')
# div_accordion_container = div_curriculum_structure.find_all('div', {'name': 'ACCORDION_CONTAINER'})
# for div in div_accordion_container:
#     essential_units = div.find('strong')
#     div_subjects = div.find_all('div', {'filter': 'subject'})
#     for div2 in div_subjects:
#         div_grey_500 = div2.find_all('div', {'color': 'grey._500'})
#         if "essential" in essential_units.text.lower():
#             ess.append(div_grey_500[0].text)
#         elif "elective" in essential_units.text.lower():
#             elec.append(div_grey_500[0].text)
# print(title)
# print(ess)
# print(elec)
# all_ess.append(ess)
# all_elec.append(elec)
# all_title.append(title)



In [83]:
# Create a DataFrame from the lists
data = pd.DataFrame({
    'Title': all_title,
    'Code': all_majors,
    'Essential Units': all_ess,
    'Elective Units': all_elec,
})

# Define the output Excel file path
output_file = 'major_unit.xlsx'

# Export the DataFrame to Excel
data.to_excel(output_file, index=False)

print('Data exported to Excel successfully.')

Data exported to Excel successfully.


In [78]:
print(len(all_majors))

73


In [79]:
print(len(all_ess))

86


In [91]:
# Read the Excel file into a DataFrame
data = pd.read_excel('major_unit.xlsx')

# Filter out rows with 'N/A' values
data = data.dropna()

# Create an empty list to store the transformed data
transformed_data = []

# Iterate over each row in the DataFrame
for _, row in data.iterrows():
    title = row['Title']
    code = row['Code']
    
    # Extract essential units and add them to the transformed data list
    ess_units = row['Essential Units']
    if ess_units:
        for ess_unit in eval(ess_units):
            transformed_data.append((title, code,'Essential units',  ess_unit))
    
    # Extract elective units and add them to the transformed data list
    elec_units = row['Elective Units']
    if elec_units:
        for elec_unit in eval(elec_units):
            transformed_data.append((title, code,'Elective units',  elec_unit))

# Create a DataFrame from the transformed data list
transformed_df = pd.DataFrame(transformed_data, columns=['Major Name', 'Major Code', 'Course Type', 'Course Code'])

# Print the transformed DataFrame

print(transformed_df)

                         Title Major Code      Course Type Course Code
0    Human Resource Management    N000191  Essential units    MGMT1007
1    Human Resource Management    N000191  Essential units    MGMT2022
2    Human Resource Management    N000191  Essential units    MGMT2040
3    Human Resource Management    N000191  Essential units    MGMT2050
4    Human Resource Management    N000191  Essential units    MGMT3007
..                         ...        ...              ...         ...
559                 CA Studies    R000253  Essential units    ACCG8221
560                 CA Studies    R000253  Essential units    ACCG8223
561                 CA Studies    R000253  Essential units    ACCG8224
562                 CA Studies    R000253  Essential units    ACCG8225
563                 CA Studies    R000253  Essential units    ACCG8226

[564 rows x 4 columns]


In [93]:
# Define the output Excel file path
output_file = 'major_unit_transform.xlsx'

# Export the DataFrame to Excel
transformed_df.to_excel(output_file, index=False)

print('Data exported to Excel successfully.')

Data exported to Excel successfully.


In [ ]:
## HANDBOOK MAJOR UNIT SCRAPING END ##

In [105]:
# Read the Excel file
df = pd.read_excel("major_unit_transform.xlsx")

# Remove duplicate course codes and get corresponding course names
unique_courses = df.drop_duplicates(subset="Course Code")
course_codes = unique_courses["Course Code"]
course_names = unique_courses["Major Name"]

all_course_code = []
all_course_name = []
period = []
error_list = []
# Scrape the website for each course code
for course_code, course_name in zip(course_codes, course_names):
    url = f"https://coursehandbook.mq.edu.au/2023/units/{course_code}"
    driver.get(url)
    time.sleep(0.3)
    html = driver.page_source
    # Parse the HTML content using BeautifulSoup
    soup = BeautifulSoup(html, "html.parser")

    # Course Name
    div_academic_banner = soup.find('div', id='academic-item-banner')
    if div_academic_banner:
        academic_banner_h2 = div_academic_banner.find("h2")
        # Print the text of the <h2> element
        title = academic_banner_h2.text.strip().split("-")[1].strip()
    else:
        title = ''

    # Find the div with the ID "Offerings"
    div_offerings = soup.find("div", id="Offerings")

    if div_offerings:
        # Find all div elements within the div_offerings element
        div_elements = div_offerings.find_all("div")
        print(len(div_elements))

        if len(div_elements) >= 2:
            # Get the second div element
            second_div = div_elements[3]
            sessions = second_div.get_text(strip=True).split("Session ")
            formatted_sessions = "\n".join(
                ["Session " + s for s in sessions if s.strip()]
            )
            print(formatted_sessions)
            period.append(formatted_sessions)
            all_course_code.append(course_code)
            all_course_name.append(title)

    # # Process the response as needed
    # # For example, you can print the course code, course name, and response status code
    print(f"Course Code: {course_code}")
    print(title)
    print("------------------------------")

## EXPORT TO EXCEL FILE
# Assuming you have the following lists:
# Create a DataFrame with the data
data = {
    "Course Code": all_course_code,
    "Course Name": all_course_name,
    "Period": period,
}
df = pd.DataFrame(data)

# Export the DataFrame to an Excel file
filename = "course_name_major.xlsx"
sheet_name = "course_name_major"
df.to_excel(filename, sheet_name=sheet_name, index=False)


4
Session 1, In person-scheduled-weekday, North Ryde
Session 1, Online-scheduled-weekday
Session 2, In person-scheduled-weekday, North Ryde
Session 2, Online-scheduled-weekday
Course Code: MGMT1007
Introduction to Human Resource Management
------------------------------
4
Session 2, In person-scheduled-weekday, North Ryde
Session 2, Online-scheduled-weekday
Course Code: MGMT2022
Foundations in Human Resources Law
------------------------------
4
Session 1, In person-scheduled-weekday, North Ryde
Session 1, Online-scheduled-weekday
Session 2, In person-scheduled-weekday, North Ryde
Session 2, Online-scheduled-weekday
Course Code: MGMT2040
Workplace Relations
------------------------------
4
Session 1, In person-scheduled-weekday, North Ryde
Session 1, Online-scheduled-weekday
Session 2, In person-scheduled-weekday, North Ryde
Session 2, Online-scheduled-weekday
Course Code: MGMT2050
Staffing Organisations
------------------------------
4
Session 1, In person-scheduled-weekday, North Ryd

In [100]:
print(all_course_name)

['Human Resource Management', 'Human Resource Management', 'Human Resource Management', 'Human Resource Management', 'Human Resource Management', 'Human Resource Management', 'Human Resource Management', 'Human Resource Management', 'Human Resource Management', 'International Business', 'International Business', 'International Business', 'International Business', 'International Business', 'International Business', 'International Business', 'International Business', 'Accounting', 'Accounting', 'Accounting', 'Accounting', 'Accounting', 'Accounting', 'Accounting', 'Accounting', 'Accounting', 'Business Analytics', 'Business Analytics', 'Business Analytics', 'Business Analytics', 'Business Analytics', 'Business Analytics', 'Business Information Systems', 'Business Information Systems', 'Business Information Systems', 'Business Information Systems', 'Business Information Systems', 'Business Information Systems', 'Cyber Security Governance', 'Cyber Security Governance', 'Cyber Security Govern

In [106]:
# Read the first Excel file into a DataFrame
df1 = pd.read_excel('course_name_major.xlsx')

# Read the second Excel file into a DataFrame
df2 = pd.read_excel('course_name.xlsx')

# Concatenate the two DataFrames vertically
concatenated_df = pd.concat([df1, df2])

# Print the concatenated DataFrame
print(concatenated_df)

    Course Code                                        Course Name  \
0      MGMT1007          Introduction to Human Resource Management   
1      MGMT2022                 Foundations in Human Resources Law   
2      MGMT2040                                Workplace Relations   
3      MGMT2050                             Staffing Organisations   
4      MGMT3007            International Human Resource Management   
..          ...                                                ...   
582    EDST8280   Contemporary Approaches to Professional Practice   
583    PICT8050                                 Strategic Thinking   
584    PICT8921           Australia's Strategic and Defence Policy   
585    PICT8951                Simulation in Security and Strategy   
586    PICT8002  Information Warfare, Disinformation and Propag...   

                                                Period  
0    Session 1, In person-scheduled-weekday, North ...  
1    Session 2, In person-scheduled-weekday, 

In [107]:
# Export the DataFrame to an Excel file
filename = "course_name_all.xlsx"
sheet_name = "course_name_all"
concatenated_df.to_excel(filename, sheet_name=sheet_name, index=False)